In [ ]:
%%time

import tensorflow as tf
import tensorflow_datasets as tfds

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

batch_size = 128
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)

In [ ]:
# CPU times: user 34.9 s, sys: 24.5 s, total: 59.4 s
# Wall time: 2min 1s

# Miniforge setup
# CPU times: user 38.1 s, sys: 23.7 s, total: 1min 1s
# Wall time: 2min 1s

# system python 3.8 /usr/bin/python3 w/ tensorflow-macos only - reports no gpu, but 8 cores busy training
# CPU times: user 29min 54s, sys: 3min 10s, total: 33min 4s
# Wall time: 5min 37s

# system python 3.8 /usr/bin/python3 w/ tensorflow-macos & tensorflow-metal - reports gpu, and gpu busy while training
# CPU times: user 39.1 s, sys: 24.4 s, total: 1min 3s
# Wall time: 1min 51s

# homebrew python 3.9.10 w/ tensorflow-macos & tensorflow-metal - reports gpu, and gpu busy while training
# CPU times: user 36.8 s, sys: 23.4 s, total: 1min
# Wall time: 1min 49s

# 9.10.2022 MacBook Air M1
# miniconda python 3.10.6 w/ tensorflow-macos & tensorflow-metal - reports gpu, and gpu busy while training
# CPU times: user 34.7 s, sys: 23.1 s, total: 57.8 s
# Wall time: 1min 46s